In [ ]:
import pandas as pd
from translate.storage.tmx import tmxfile
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Data available at https://opus.nlpl.eu/OpenSubtitles-v2018.php
# https://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/tmx/en-pt.tmx.gz
with open("../../data/en-pt.tmx", 'rb') as fin:
    tmx_file = tmxfile(fin, 'en', 'pt')

In [ ]:
# put tmx file into a pandas dataframe
df = pd.DataFrame(columns=['source', 'target'])
for unit in tmx_file.units:
    df = df.append({'source': unit.source, 'target': unit.target}, ignore_index=True)
df

In [ ]:
df

In [ ]:
# Save it to save time
df.to_csv("../../data/corpus/en-pt.tsv", index=False, sep='\t')

In [ ]:
# Load it
df = pd.read_csv("../../data/corpus/en-pt.tsv", sep='\t')
df

In [ ]:
# Create a smaller dataset for testing
df = df.sample(frac=0.1)
df.to_csv("../../data/corpus/en-pt-small.tsv", index=False, sep='\t')

In [20]:
# Load it
df_small = pd.read_csv("../../data/corpus/en-pt-small.tsv", sep='\t')
df_small

,source,target
0,"No, I know I saw...","Não, eu sei que vi..."
1,It is written that only the high priestess of ...,Está escrito que só a sacerdotisa pode escolhe...
2,"Lie down a second. - Let me get comfortable, p...",- Vamo-nos pôr bem.
3,- Sexpot?,- Sensual!
4,- What do I do with it?,- E que faço com ele?
...,...,...
212415,He gave me 10.000 Lire!,ele deu-me 10.000 Liras!
212416,"On the other side of the mountain, New Jerusalem.",Do outro lado da montanha fica Nova Jerusalém.
212417,I saw you!,Eu vi-os!
212418,Your father took you to baseball and football ...,O teu pai levava-te ao basebol e ao futebol?


In [29]:
len(df_small["source"].str.split(" - ").explode().to_list())

215576

In [28]:
len(df_small["target"].str.split(" - ").explode().to_list())

214987

In [25]:
# Filter out nonsense in the sentences
for col in df_small.columns:
    # Remove the trailing "- " in the sentences but only if they are at the beginning of the sentence
    df_small[col] = df_small[col].str.replace("^- ", "", regex=True).str.lower()
    # Separate the sentences that can be split with " - "
df_small = pd.DataFrame(df_small["source"].str.split(" - ").explode(), df_small["target"].str.split(" - ").explode())
df_small

ValueError: cannot reindex on an axis with duplicate labels

In [12]:
[s.replace("- ", "\n") for s in df_small.target.to_list()[:10]]

['não, eu sei que vi...',
 'está escrito que só a sacerdotisa pode escolher o companheiro.',
 '\nvamo-nos pôr bem.',
 '\nsensual!',
 '\ne que faço com ele?',
 '\nmas o william claiborne não sobreviveu ao confronto do ok corral?',
 'diga-nos que está tudo bem !',
 '\ntaybor, se pudéssemos falar sobre...',
 'tenho sua nave capitão.',
 'ele não te traí, césar.']